### Подготовка: установка зависимостей, импорты и загрузка/выгрузка внешних переменных

In [1]:
!pip install -r requirements.txt

In [2]:
import requests
import os
import json
import langchain

%reload_ext dotenv
%dotenv

api_key = os.getenv('YAGPT_API_KEY')
folder_id = os.getenv('YA_FOLDER_ID')

### Класс для взаимодействия с моделью - заполняет поля, отправляет запрос и получает ответ

In [3]:
from typing import Optional, List, Mapping, Any
from langchain.llms.base import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun

class YandexLLM(LLM):
    api_key: str = None
    folder_id: str = None
    max_tokens : int = 1500
    temperature : float = 1
    instruction_text : str = None
    
    # def __init__(self, api_key, folder_id, instruction_text, prompt=None, max_tokens=1000, temperature=0.6):
    #     self.api_key = api_key
    #     self.folder_id = folder_id
    #     self.headers = {'Authorization': 'Api-key ' + self.api_key,
    #                     "x-folder-id" : self.folder_id }
    #     self.instruction_text = instruction_text
    #     self.prompt = prompt
    #     self.max_tokens = max_tokens
    #     self.temperature = temperature
        
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:

        if stop is not None:
            raise ValueError("stop kwargs are not permitted.")

        headers["Authorization"] = f"Api-key {self.api_key}"

        req = {
            "model": "general",
            "instruction_text": self.instruction_text,
            "request_text": self.prompt,
            "generation_options": {
                "max_tokens": self.max_tokens,
                "temperature": self.temperature
            }
            #"requestText": "Привет!"
        }

        print("Sending request to TextGeneration.instruct API")

        res = requests.post("https://llm.api.cloud.yandex.net/llm/v1alpha/instruct",
          headers=headers, json=req).json()

        print("Received response from Yandex LLM API.")

        return res['result']['alternatives'][0]['text']

        # res = requests.post("https://llm.api.cloud.yandex.net/llm/v1alpha/instruct",
        #                     headers=self.headers, json=req).json()
        # print("Received response from Yandex LLM API.")
        
        # return res

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        """Get the identifying parameters."""
        return {"model_name": "YandexGPT_RAGbot"}
    
    @property
    def _llm_type(self) -> str:
        return "yagpt"


### Подготовка переменных для промпта

In [4]:
page = '''Инструкция по регистрации аккаунта на сервисе MyApp:

### Шаг 1: Переход на веб-сайт MyApp
1. Откройте любой веб-браузер на вашем компьютере или мобильном устройстве.
2. Введите адрес MyApp в адресной строке браузера и нажмите Enter, чтобы перейти на сайт.

### Шаг 2: Нахождение кнопки "Регистрация"
1. После загрузки сайта MyApp, просмотрите домашнюю страницу и найдите кнопку "Регистрация" или "Создать аккаунт". Обычно эта кнопка располагается в верхнем правом углу страницы или на главной странице.

### Шаг 3: Заполнение регистрационной формы
1. Щелкните по кнопке "Регистрация". Вас перенаправит на страницу с регистрационной формой.
2. Заполните поля формы, предоставив необходимую информацию, такую как:
   - Имя пользователя: Уникальное имя, которое будет использоваться для входа на сервис.
   - Пароль: Безопасный пароль, состоящий из букв, цифр и специальных символов.
   - Адрес электронной почты: Действующий адрес электронной почты для подтверждения аккаунта.
   - Другие данные: Возможно, потребуется указать дополнительную информацию, такую как дата рождения, пол и т. д.

### Шаг 4: Подтверждение регистрации
1. После заполнения всех полей формы, прочтите условия использования и политику конфиденциальности, если они доступны.
2. Если вы согласны с условиями, отметьте соответствующее поле или нажмите кнопку "Согласен" или "Зарегистрироваться".
3. Система отправит на ваш электронный адрес письмо с подтверждением. Откройте свою почту и следуйте инструкциям для завершения процесса регистрации.

### Шаг 5: Вход в свой аккаунт
1. После успешной регистрации возвращайтесь на главную страницу MyApp.
2. Введите свое имя пользователя и пароль в соответствующие поля на странице входа.
3. Нажмите кнопку "Войти" или "Вход", чтобы войти в свой аккаунт.

Поздравляю! Теперь у вас есть аккаунт на сервисе MyApp, и вы можете начать пользоваться его функциями. Не забудьте проверить свой профиль и настройки, чтобы убедиться, что все настроено по вашему вкусу.'''

query = "Куда должно прийти письмо с подтверждением регистрации?"

### Создание промпта и цепочки для LLM

In [7]:
from langchain.chains import StuffDocumentsChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Instruction for YandexGPT
instruction = "Представь, что ты сотрудник компании MyApp. Отвечай на вопросы вежливо и по мере своих возможностей."

# It accepts a set of parameters from the user that can be used to generate a prompt 
# Example for understanding:
# prompt = PromptTemplate(input_variables=["foo"], template="Say {foo}") 
# If foo="Hello", Result: "Say Hello"
document_template = PromptTemplate(
    input_variables=["page_content"], #A list of the names of the variables, that document prompt template expect (as a variable type).
    template="{page_content}" # The document prompt template.
)

# The variable name where to put document in final prompt.
document_variable_name = "context"

# Template for prompt
template_override = """
    Пожалуйста, посмотри на текст ниже и ответь на вопрос, используя информацию из этого текста.
    Текст:
    -----
    {context}
    -----
    Вопрос:
    {query}
"""
# create prompt
prompt = langchain.prompts.PromptTemplate(
    input_variables=["context", "query"],
    template=template_override
)

# LLM object 
llm = YandexLLM(api_key=api_key, folder_id=folder_id, instruction_text=instruction)

# create chain step 1: llm and prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

# create chain step 2: llm, prompt. And add: document, document variable 
chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_prompt=document_template,
    document_variable_name=document_variable_name
)

In [10]:
chain.run(input_documents=page, query=query)

AttributeError: 'str' object has no attribute 'page_content'